In [6]:
import numpy as np
import math
import random
import datetime
import statistics
import pandas as pd
import seaborn as sn
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics  
from sklearn.model_selection import KFold
from sklearn.tree import export_graphviz
from IPython.display import Image  
import pydotplus
from sklearn.model_selection import GridSearchCV
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')



In [7]:
dates = []
with open('./data/dates.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        dates.append(x)

In [19]:
percisions,totalgains=[],[]
def test_classifier(startindex , endindex, percent):
    for i in range(startindex, endindex):
        print(f"date is {dates[i]}")
        daily_gain,counter= 0,0
        # the current date (the training data) df cleaned
        date_df = pd.read_csv(f"./data/dates{percent}/{dates[i]}.csv")
        date_df = date_df.loc[:, ~date_df.columns.str.contains('^Unnamed')]
        
        # the next day (the validation data) date df cleaned
        next_date_df = pd.read_csv(f"./data/dates{percent}/{dates[i + 1]}.csv")
        next_date_df = next_date_df.loc[:, ~next_date_df.columns.str.contains('^Unnamed')]
        
        # the next next day to get gains and losses 
        results_df = pd.read_csv(f"./data/dates{percent}/{dates[i + 2]}.csv")
        results_df = results_df.loc[:, ~results_df.columns.str.contains('^Unnamed')]
        
        # the targets of each date to confirm models results.
        target_df = pd.read_csv(f"./data/targets/Targets{percent}.csv")
        
        #train validation split 
        x_train = date_df.drop(['ticker'], axis=1)
        x_validate = next_date_df.drop(['ticker'], axis=1) 
        y_train = target_df[dates[i+1]] 
        y_validate = target_df[dates[i+2]] 
        
        #scaling the data
#         scaler = preprocessing.StandardScaler()
#         X_train = scaler.fit_transform(x_train)
#         x_validate = scaler.fit_transform(x_validate)

        #define model and run data through it 
        model = QuadraticDiscriminantAnalysis()
        model.fit(x_train, y_train)
        y_pred = model.predict(x_validate)

        #calculate daily positions and daily gain
        for j in range(len(y_pred)):
            if y_pred[j] == True:
                counter = counter + 1
#                 print(f"gain : {round(results_df.at[j, 'Close Change']*100,2)} \t stock : {results_df.at[j, 'ticker']}")
                daily_gain = daily_gain + round(results_df.at[j, 'Close Change']*100,2)
        #print daily results (precision, recall , acc , daily gain and )
        print("--------------------------------------------")   
        percisions.append(metrics.precision_score(y_validate, y_pred))
        if counter == 0 :
            totalgains.append(1)
        else:
            totalgains.append(1+ (daily_gain/counter)/100)
        print(f"precision: {metrics.precision_score(y_validate, y_pred)}")
        print(f"recall: {metrics.recall_score(y_validate, y_pred)}")
        print(f"accuracy: {metrics.accuracy_score(y_validate, y_pred)}")
        if counter == 0:
            print(f"daily gain: {daily_gain/counter}\n")
        
#         print(f"daily gain: {daily_gain}\n") 
        print(pd.DataFrame(metrics.confusion_matrix(y_validate, y_pred)))
        print("--------------------------------------------")

    print("--------------------------------------------")
    print(f"avg percision:{sum(percisions) / len(percisions)}") 
    totP = 1
    for gain in totalgains:
        totP = totP*gain
    print(f"the total profit made from this classifier {totP}")
    print("--------------------------------------------")
test_classifier(450,480,1.5)

date is 2021-02-10
--------------------------------------------
precision: 0.23279352226720648
recall: 1.0
accuracy: 0.23279352226720648
daily gain: 0.57746963562753

   0    1
0  0  379
1  0  115
--------------------------------------------
date is 2021-02-11
--------------------------------------------
precision: 0.0
recall: 0.0
accuracy: 0.8016194331983806


ZeroDivisionError: division by zero